<a href="https://colab.research.google.com/github/BRV12G/Final_year_Project/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://chatgpt.com/share/6777b6a7-1af8-800b-a09e-1ca14f0df45c

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor  # Importing KNeighborsRegressor

# Load dataset
# Replace 'file_path.csv' with your actual dataset file path
data = pd.read_csv('/content/nutrition_dataset_with_fiber_water_intake (1).csv')

# 1. Drop 'Person ID' and 'Health Status' columns
data = data.drop(columns=['Person ID', 'Health Status'])

# 2. Check for missing values and handle them
# Fill missing numerical values with the mean, and categorical values with the mode
for column in data.columns:
    if data[column].dtype == 'object':
        data[column].fillna(data[column].mode()[0], inplace=True)
    else:
        data[column].fillna(data[column].mean(), inplace=True)

# 3. Handle outliers using the IQR method
def handle_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[col] = np.clip(df[col], lower_bound, upper_bound)
    return df

numerical_columns = [
    'Age', 'Sleep Duration', 'Weight (kg)', 'Height (cm)', 'Systolic',
    'Diastolic', 'Heart Rate', 'Daily Steps', 'BMI Values'
]
data = handle_outliers(data, numerical_columns)

# Separate inputs and outputs
input_columns = numerical_columns + [
    'Gender', 'Occupation', 'Quality of Sleep', 'Activity Level',
    'Stress Level', 'Blood Pressure Category', 'BMI Class'
]
output_columns = [
    'Calories (kcal)', 'Carbohydrates (g)', 'Proteins (g)', 'Fats (g)',
    'Vitamin A (mcg)', 'Vitamin C (mg)', 'Vitamin D (mcg)', 'Sodium (mg)',
    'Potassium (mg)', 'Magnesium (mg)', 'Iron (mg)', 'Zinc (mg)',
    'Fiber Intake (g)', 'Water Intake (L)'
]
X = data[input_columns]
y = data[output_columns]

# 4. Preprocess categorical and numerical data
# Binary encode Gender
X['Gender'] = X['Gender'].map({'Male': 0, 'Female': 1})

# Label encode categorical columns
label_columns = ['Quality of Sleep', 'Activity Level', 'Stress Level', 'Blood Pressure Category', 'BMI Class']
label_encoders = {col: LabelEncoder() for col in label_columns}
for col in label_columns:
    X[col] = label_encoders[col].fit_transform(X[col])

# One-hot encode Occupation
X = pd.get_dummies(X, columns=['Occupation'], drop_first=True)

# Standardize and normalize numerical columns
scaler = Pipeline(steps=[
    ('standardize', StandardScaler()),
    ('normalize', MinMaxScaler())
])
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# 5. Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Train the model using KNN Regressor
knn_model = KNeighborsRegressor(n_neighbors=5)  # Using KNN Regressor with 5 neighbors
multioutput_model = MultiOutputRegressor(knn_model)
multioutput_model.fit(X_train, y_train)

# 7. Evaluate the model
y_pred = multioutput_model.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred))
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

# 8. Save the model for later use
import joblib
joblib.dump(multioutput_model, 'knn_multioutput_model.pkl')


<ipython-input-1-e9e8bec31797>:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(data[column].mode()[0], inplace=True)
<ipython-input-1-e9e8bec31797>:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

Mean Absolute Error (MAE): 16.936356823430508
Mean Squared Error (MSE): 3436.217731837648
R² Score: 0.9295552771630547


['knn_multioutput_model.pkl']